<a href="https://colab.research.google.com/github/ebatty/MathToolsforNeuroscience/blob/master/Week3/Week3Tutorial1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Week 3: Linear Algebra III

# Tutorial 1

# [insert your name]

**Important reminders**: Before starting, click "File -> Save a copy in Drive". Produce a pdf for submission by "File -> Print" and then choose "Save to PDF".

To complete this tutorial, you should have watched Videos 3.1 and 3.2.

You can change the theme of a jupyter notebook between dark and light (black vs white background): Go to Tools -> Settings -> theme. Specify which theme you are using below so the plots change accordingly. I think dark mode is prettier/nicer.


In [ ]:
your_theme = 'dark' # change this to light for plots that work well with white background

In [ ]:
# @title 
# @markdown Imports

# Imports
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import scipy.linalg
from matplotlib.animation import FuncAnimation

# Plotting parameters
from IPython.display import HTML
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams.update({'font.size': 22})

In [ ]:
# @title 
# @markdown Plotting functions
if your_theme == 'dark':  
  plt.style.use(['dark_background'])
  classic = 'w'
else:
  classic = 'k'

def plot_eig_vec_transform(W):
  vec_names = ['a', 'b','c','d','e','f','g', 'h']

  _, vecs = np.linalg.eig(W)
  vecs = vecs.T

  fig, axes = plt.subplots(1, 2, figsize=(10, 5))
  colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

  for i in range(2):
    axes[i].set(xlim=[-3.5, 3.5], ylim=[-3.5,3.5])
    axes[i].axis('Off')
    axes[i].plot([0, 0], [-3.5, 3.5], classic, alpha=.4)
    axes[i].plot([-3.5, 3.5], [0, 0], classic, alpha=.4)

  for i_vec, vec in enumerate(vecs):    
    axes[0].arrow(0, 0, vec[0], vec[1], head_width=.2, facecolor=colors[i_vec], edgecolor=colors[i_vec], length_includes_head=True)
    axes[0].annotate(vec_names[i_vec], xy=(vec[0]+np.sign(vec[0])*.15, vec[1]+np.sign(vec[1])*.15), color=colors[i_vec])

    transformed_vec = np.matmul(W, vec)
    axes[1].arrow(0, 0, transformed_vec[0], transformed_vec[1], head_width=.2, facecolor=colors[i_vec], edgecolor=colors[i_vec], length_includes_head=True)
    axes[1].annotate(vec_names[i_vec], xy=(transformed_vec[0]+np.sign(transformed_vec[0])*.15, transformed_vec[1]+np.sign(transformed_vec[1])*.15), color=colors[i_vec])

  axes[0].set_title('Before')
  axes[1].set_title('After')

def animate_circuit_responses(u, W, xlim='default', ylim='default'):
    fig, ax = plt.subplots(1, 1, figsize=(10,10))

    # Set up axis limits
    if xlim =='default':
      xlim = [np.minimum(np.min(u), 0), np.maximum(np.max(u)+5, 0)]
    if ylim == 'default':
      ylim = [np.minimum(np.min(u), 0), np.maximum(np.max(u)+5, 0)]

    # Set up look
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    cs = plt.rcParams['axes.prop_cycle'].by_key()['color']*10
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)

    # Set up tracking textz
    tracker_text = ax.text(.5, .9, "", color='w', fontsize=20, verticalalignment='top', horizontalalignment='left', transform=ax.transAxes)

    # Plot eigenvectors
    eigvals, eigvecs = np.linalg.eig(W)


    if np.abs(eigvals[0]) < np.abs(eigvals[1]):
      lc1 = 'c'
      lc2 = 'g'
    else:
      lc1 = 'g'
      lc2 = 'c'

    ax.plot(np.arange(-10000, 10000)*eigvecs[0, 0], np.arange(-10000, 10000)*eigvecs[1, 0],lc1, alpha=.5)
    ax.plot(np.arange(-10000, 10000)*eigvecs[0, 1], np.arange(-10000, 10000)*eigvecs[1, 1], lc2, alpha=.5)


    # Set up scatter 
    scatter = ax.scatter([], [], alpha=1, c=cs[0]) 

    # Update function during animation
    def update(num):
        scatter.set_offsets(u[:, :(num+1)].T)
        scatter.set_facecolors(cs[:(num)])

        track_string = '   t = '+str(num)+'\n$N_1$ = '+str(round(u[0,num], 2))+' \n$N_2$ = '+str(round(u[1,num], 2))+''
        tracker_text.set_text(track_string)

    
    # Make/display animation
    ani = FuncAnimation(fig, update, T, interval=10000/T, blit=False)
    plt.close(fig)
    return ani, eigvals, eigvecs
    

# Key concept review & coding tips

## Eigenvectors & eigenvalues


*   An eigenvector is a vector that doesn't change direction in a transformation ($\bar{v}$ below).
*   The associated eigenvalue is the factor by which the eigenvector is scaled ($\lambda$ below). \\
$$A\bar{v} = \lambda\bar{v} $$

*   `np.linalg.eig` gives you the eigenvalues and eigenvectors of a matrix.
*  Eigenvalues are more specifically defined than eigenvectors: for an eigenvector $\bar{v}$, $-2\bar{v}$ and $6\bar{v}$ are also eigenvectors. In other words, there are an infinite number of eigenvectors for a given eigenvalue that lie along the same line. For this reason, we often use unit eigenvectors (although this does not account for flipping between negative/positive). 
*  Additionally, sometimes all of the vectors in a plane (or more in higher D space) can be eigenvectors for a given eigenvalue. This will not always be obvious from the outputs of `np.linalg.eig`.
*  You will not necessarily be able to form a basis for a space from the eigenvectors.
*  We solve for the eigenvalues of matrix $B$ by solving det($B$ - $\lambda I$) = 0.
* Complex eigenvalues exist and have associated complex eigenvectors - a matrix with complex eigenstuff performs some kind of rotation.
*  Nonsquare matrices do not have eigenvalues.
* An n x n matrix has n eigenvalues but they could be real or complex and they are not necessarily distinct (meaning a matrix could have two eigenvalues of 2, with different associated eigenvectors).

## Eigenstuff in neural circuits

*  We can model a neural circuit as a discrete dynamical system. If $\bar{u}_t$ is the vector of neural activity at time $t$ and $W$ is the synaptic weight matrix, then:
$$\bar{u}_{t+1} = W\bar{u}_t $$ 
*  The eigenvalues/eigenvectors of W can help us understand the system and what will happen given initial responses $\bar{u}_0$. 

Assuming $W$ is diagonalizable (aka we can form an eigenbasis, aka we have N linearly independent eigenvectors):
* If the absolute values  of all the eigenvalues of W are greater than 1, the system will diverge to infinity.
* If the absolute values  of all the eigenvalues of W are less than 1, the system will converge to the origin.
* If the eigenvalues are all equal to 1, the system will stay at the initial state.


# Exercise 1: Eigenvectors & transformations

## A) Identifying eigenvectors

Below we plot some vectors before a linear transformation by a matrix and after. Which of the vectors are eigenvectors?

In [ ]:
# @title
# @markdown Execute this cell to visualize vectors

vec_names = ['a', 'b','c','d','e','f','g', 'h']
vecs = np.array([[0, 1], 
                 [1/np.sqrt(2), 1/np.sqrt(2)],
                 [1, 0],
                 [1/np.sqrt(2), -1/np.sqrt(2)],
                 [0, -1], 
                 [-2/np.sqrt(5), -1/np.sqrt(5)],
                 [-1, 0], 
                 [-2/np.sqrt(5), 1/np.sqrt(5)], 
                 ])
W = np.array([[1.4, 1.2], [0, 1]])

fig, axes = plt.subplots(1, 2, figsize=(10, 5))
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

for i in range(2):
  axes[i].set(xlim=[-2, 2], ylim=[-2,2])
  axes[i].axis('Off')

for i_vec, vec in enumerate(vecs):    
  axes[0].arrow(0, 0, vec[0], vec[1], head_width=.2, facecolor=colors[i_vec], edgecolor=colors[i_vec], length_includes_head=True)
  axes[0].annotate(vec_names[i_vec], xy=(vec[0]+np.sign(vec[0])*.15, vec[1]+np.sign(vec[1])*.15), color=colors[i_vec])

  transformed_vec = np.matmul(W, vec)
  axes[1].arrow(0, 0, transformed_vec[0], transformed_vec[1], head_width=.2, facecolor=colors[i_vec], edgecolor=colors[i_vec], length_includes_head=True)
  axes[1].annotate(vec_names[i_vec], xy=(transformed_vec[0]+np.sign(transformed_vec[0])*.15, transformed_vec[1]+np.sign(transformed_vec[1])*.15), color=colors[i_vec])

axes[0].set_title('Before')
axes[1].set_title('After');

**Your text answer**

## B) Describing transformations with eigenvectors

Last week, we learned how to think about linear transformations in terms of where the unit vectors end up. We can also think about them in terms of eigenvectors. Just by looking at eigenvectors before and after a transformation, can you describe what the transformation is in words? 

Note that I show an eigenvector for every eigenvalue. The x/y limits do not change in before vs after (so eigenvectors are showed scaled by the eigenvalues).


Here are some transformation words to jog your memory: contraction, expansion, horizontal vs vertical, shear, projection onto an axis, reflection.

In [ ]:
# @title
# @markdown Execute this cell to visualize vectors

W = np.array([[3, 0], [0, 1]])
plot_eig_vec_transform(W)


**Your text answer for plot above**

In [ ]:
# @title
# @markdown Execute this cell to visualize vectors

W = np.array([[0, 1], [1, 0]])
plot_eig_vec_transform(W)

**Your text answer for plot above**

# Exercise 2: Eigenstuff of a matrix squared

This is a guided proof. Consider the matrix $A$ with eigenvalues $\lambda$. We want to answer the following question: what are the eigenvectors and eigenvalues of $A^2$?

i) Write down the relationship between $A$ and its eigenvalues $\lambda$ and eigenvectors $\bar{v}$.

**Your math answer**

ii) Multiply both sides of the equation by $A$. Is there anything on the right-hand side that you can replace $A\bar{v}$ with?

**Your math answer**

iii) What are the eigenvalues and eigenvectors of $A^2$?


**Your math/text answer**

# Exercise 3: Complex computations

We saw in Video 3.2 that eigenvalues can be complex - let's convince ourselves of this. Consider the following rotation matrix:

$$B = 
\begin{bmatrix}
0 & -1 \\
1 & 0 \\
\end{bmatrix} $$

Calculate the eigenvalues of this matrix using the method we learned in class (use det($B$ - $\lambda I$) = 0 to solve for $\lambda$).

Discuss (no need to explicitly answer): why does it make sense that a rotation has no real eigenvalues?

**Your math answer**

# Exercise 4: Neural circuit dynamics

Consider a two neuron circuit with the following synaptic connectivity matrix:

$$W = \begin{bmatrix}
1 & 3 \\
4 & 2 \\
\end{bmatrix} $$

## A) Drawing the circuit

Draw this two neuron circuit with the labeled synaptic weights.

**Your drawn diagram here**

## B) Computing eigenstuff with code
Calculate the eigenvalues and the eigenvectors of W using `np.linalg.eig` and print them. Make sure you understand what the outputs are! 

In [ ]:
eigvals, eigvecs = ...

Prove to yourself that the computer is right. Multiply W with one of your computed eigenvectors and make sure it equals the associated eigenvalue times that eigenvector. Print True if these resulting vectors are equal.

In [ ]:
# Your code here

Do you expect the system to converge to the origin or diverge to infinity? Why?


**Your text answer here**

## C) Modeling the circuit

Complete the function below for computing $\bar{u}$ over time.  

You will then see an animation of $\bar{u}$ over time using a provided function `animate_circuit_responses`. The blue/green lines in this animation show the span of the eigenvectors: green represents the eigenvectors with the largest magnitude eigenvalue. Feel free to change the axis limits to see interesting parts of space (xlim/ylim arguments).


Assume an input vector (initial condition) $\bar{u}_0 = \begin{bmatrix}
1 \\
1  \\
\end{bmatrix}$. 

In [ ]:
def circuit_implementation(W, u0, T):
  """ Simulate the responses of N neurons over time given their connections

  Args:
    W (ndarray): weight matrix of synaptic connections, should be N x N
    u0 (ndarray): initial condition or input vector, should be N,
    T (scalar): number of time steps to run simulation for

  Returns:
    u (ndarray): the neural responses over time, should be N x T

  """

  # Compute the number of neurons
  N = W.shape[0] 

  # Initialize empty response array and initial condition
  u = np.zeros((N, T))
  u[:, 0]  = u0

  ## Your code here! 
  # You should loop over time steps and compute u(t+1) based on u(t), and store in 
  # appropriate column of u (each column is a time step)

  # Comment out below when you have finished
  raise NotImplementedError('Student code here')

  return u


W = np.array([[1, 3], [4, 2]])
u0 = np.array([1, 1])
T = 6

u = circuit_implementation(W, u0, T)

ani, eigvals, eigvecs = animate_circuit_responses(u, W, xlim=[-720, 720], ylim=[-720, 720]) 
ani

What are the activities of $N_1$ and $N_2$ after 5 timesteps? 

**Your math/text answer here**

What are the activities of $N_1$ and $N_2$ after 5 timesteps if you start with initial condition $\bar{u}_0 = \begin{bmatrix}
-3 \\
-1  \\
\end{bmatrix}$. Note that we are allowing our neural activities to be negative.

**Your math/text answer here**

## D) Mixed eigenvalues

In Video 3.2, **I** mostly discussed matrices that had either all eigenvalues with absolute value greater than 1 or all eigenvalues with absolute value less than 1. Let's explore a new weight matrix that has one eigenvalue above 1 and one below:

$$W = \begin{bmatrix}
1 & .2 \\
.1 & 1 \\
\end{bmatrix} $$

Use the animation below to answer i-iii by changing the initial condition.

In [ ]:
u0 = ...

W = np.array([[1, .2], [.1, 1]])
T = 25

u = circuit_implementation(W, u0, T)
ani, eigvals, eigvecs = animate_circuit_responses(u, W, xlim=[-15, 15], ylim=[-15, 15]) 
ani

i) Look at what happens when the initial inputs $\bar{u}_0 = \begin{bmatrix}
1 \\
2  \\
\end{bmatrix}$. What do you notice about the activities of $N_1$ and $N_2$ and how they relate to the eigenvectors?

**Your text answer**

ii) Now look at the initial condition $\bar{u}_0 = \begin{bmatrix}
-1 \\
1  \\
\end{bmatrix}$. What happens? Describe in words the evolution of $N_1$ and $N_2$ and how it relates to the eigenvectors.

**Your text answer**

iii) Can you design an initial condition so that the system converges to the origin (the response of both neurons goes to 0)? Give such a condition if it exists.

**Your math/text answer**